In [112]:
import requests 
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import minsearch
import openai
import os
from dotenv import load_dotenv
import tiktoken
load_dotenv()

API_KEY = os.getenv('API_KEY')

In [43]:

es_client = Elasticsearch("http://localhost:9200")

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []


# Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_faq"


es_client.indices.create(index = index_name, body = index_settings)

for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)



BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course_faq/LTMDvz5CSgmEC3vENeYUdA] already exists')

In [44]:
query = "How do I execute a command in a running docker container?"

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [46]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }#,
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                }
            }
        }
    #}

    response = es_client.search(index=index_name, body=search_query)

    print(response)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit["_score"])#hit['_source'])
    
    return result_docs


## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

In [47]:
elastic_search(query)

{'took': 10, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course_faq', '_id': 'Ur2hRZABPQrPhNZcN7hS', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course_faq', '_id': 'YL2hRZABPQrPhNZcLLaT', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the 

[84.050095, 75.54128, 72.08518, 51.04628, 49.938507]

Now let's only limit the questions to `machine-learning-zoomcamp`.



Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* **How do I copy files from a different folder into docker container’s working directory?**
* How do Lambda container images work?
* How can I annotate a graph?


In [100]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

   # print(response)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit["_source"])#hit['_source'])
    
    return result_docs

In [101]:
search_results = elastic_search(query)
search_results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)


In [109]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    
    # context_template = """
    #     Q: {question}
    #     A: {text}
    #     """.strip()
    context = ""
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt



In [110]:
for doc in search_results:
    # context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    question = doc['question']
    text = doc["text"]
    context_template = f"""
    Q: {question}
    A: {text}
    """.strip()
    print(context_template)

Q: How do I debug a docker container?
    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)
Q: How do I copy files from my local machine to docker container?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani
Q: How do I copy files from a different folder into docker container’s working directory?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here

In [111]:
len(build_prompt(query, search_results=search_results))

1479


Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* **322**
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [121]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [122]:
len(encoding.encode(prompt))

322

In [124]:
decoded_prompt = ''.join([encoding.decode_single_token_bytes(token).decode('utf-8') for token in encoding.encode(prompt)])


In [125]:
decoded_prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dir